# Access Control - Inheriting Permissions

This notebook will demonstrate how a resource inherits permissions. 

- Create a resource without own *accessControlPolicyIDs* attribute
- Access the resource from another originator


## Intitialization
<div class="alert alert-block alert-info" style="border-radius: 10px;">
This notebook relies on the resources created in the exercise's previous notebooks.
<br/>

The following section imports and initializes necessary functions and configurations. 
It must be executed once for each notebook before running any of cells.</div>

In [ ]:
%run ../src/init.py

## Alice adds a \<contentInstance> resource

With the following request Alice adds a \<contentInstance> resource to the \<cnt> resource:

In [ ]:
CREATE (                                                      # CREATE request
    target                  = f'{cseRN}/aeAlice/container',   # Create the <cin> under aeAlice's <cnt>
    originator              = 'CAlice',                       # Request's originator (mandatory)
    requestIdentifier       = '123',                          # Request identifier (mandatory)
    releaseVersionIndicator = '3',                            # Request's release version indicator (mandatory)
    resourceType            = Type.ContentInstance,           # The request creates a <cin> resource
    
    # Request Content
    content = {
        "m2m:cin": {
            "rn" : "cin",                                     # Assign a nice resource name so that we can access the resource easier
            "con": "hello, world"                             # Only add some content
        }
    }
)

## Bob tries to retrieve Alice's \<contentInstance> resource

In one of the previous exercises Alice created an \<accessControlPolicy> resource that defines RETRIEVE permissions for Bob, and assigned them to the container. 

But how does this relate to the \<contentInstance>, which is only a child-resource of the \<container>?

In [ ]:
RETRIEVE (                                                       # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container/cin',  # Target resource is the <cin> under Alice's <cnt>
    originator              = 'CBob',                            # Request's originator is CBob (mandatory)
    requestIdentifier       = '123',                             # Request identifier (mandatory)
    releaseVersionIndicator = '3'                                # Request's release version indicator (mandatory)
)

Bob can access the \<contentInstance> resource, even when there is no explicit \<accessControlPolicy> resource defined. How does this work?

If we have a look at the \<contentInstance>'s specification then we will notice that this resource type does not have the *accessControlPolicyIDs* attribute defined. In this case the following rule applies.

<div class="alert alert-block alert-success" style="border-radius: 10px">
Some resource types do not have their own <i>accessControlPolicyIDs</i> attribute, but inherit the permissions or access authorizations from the parent resource. The authorization for those resources is not determined when the resources are created, but happens dynamically at runtime. 
    
This also means if the permissions for a parent resource change between two requests, then the permissions indirectly also change for its children.

Please note, that this only applies to resource types that don't have this attribute defined. It does NOT apply to resources that have the <i>accessControlPolicyIDs</i> attribute defined, but just not set. In those cases other rules apply, such as the one that, as a default, grants access to the original creator.
</div>

&nbsp;